In [1]:
from tensorflow import keras

2025-04-23 10:13:44.754962: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 10:13:44.827907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745396024.846730 3044001 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745396024.853020 3044001 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745396024.870041 3044001 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
import tensorflow as tf
from tensorflow import keras
import time

# Abilita la memoria GPU dinamica
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Prepara il dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Crea il modello CNN
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(32, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Conv2D(64, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(64, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(128, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Conv2D(128, (3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

# Compila il modello
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Inizia il timer per il training
start_time = time.time()

# Allena il modello
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
)

# Calcola il tempo totale di training
training_time = time.time() - start_time
print(f"Tempo totale di training: {training_time:.2f} secondi")

# Inizia il timer per l'inferenza
start_time = time.time()

# Esegui inferenza su tutto il test set
predictions = model.predict(x_test)

# Calcola il tempo totale di inferenza
inference_time = time.time() - start_time
print(f"Tempo totale di inferenza: {inference_time:.2f} secondi")
print(f"Tempo medio per immagine: {inference_time / len(x_test) * 1000:.2f} ms")

# Valuta il modello
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Accuratezza sul test set: {test_acc:.4f}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 62s 0us/step


/home/labeconomia/mparton/.conda/envs/keras_env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745396174.633345 3044001 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43710 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:08:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1745396180.124024 3044356 service.cc:152] XLA service 0x7f519401dec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745396180.124105 3044356 service.cc:160]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2025-04-23 10:16:20.230940: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745396180.934592 3044356 cuda_dnn.cc:529] Loaded cuDNN version 90600
I0000 00:00:1745396189.864476 3044356 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


387/391 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3246 - loss: 1.9383

2025-04-23 10:16:36.185179: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4202', 52 bytes spill stores, 52 bytes spill loads



391/391 ━━━━━━━━━━━━━━━━━━━━ 28s 37ms/step - accuracy: 0.3259 - loss: 1.9343 - val_accuracy: 0.3537 - val_loss: 1.9141
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5831 - loss: 1.1761 - val_accuracy: 0.6539 - val_loss: 0.9612
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6655 - loss: 0.9562 - val_accuracy: 0.6603 - val_loss: 0.9526
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7083 - loss: 0.8375 - val_accuracy: 0.7207 - val_loss: 0.7980
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7365 - loss: 0.7673 - val_accuracy: 0.7223 - val_loss: 0.7954
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7549 - loss: 0.7130 - val_accuracy: 0.7575 - val_loss: 0.6893
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.7685 - loss: 0.6662 - val_accuracy: 0.7814 - val_loss: 0.6356
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7870 - loss: 0.6214 - val_accuracy: 0.78

In [7]:
from tensorflow import keras

keras.backend.backend()

'tensorflow'